<a target="_blank" href="https://colab.research.google.com/github/lukebarousse/Int_SQL_Data_Analytics_Course/blob/main/3_Windows_Functions/4_Lag_Lead.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Lag / Lead

### 🥅 Analysis Goals

- What we’re going to use for this dataset to do X e.g. Use the following in order to explore a dataset on experience and salaries
    - Major topic 1
    - Major topic 2
    - Major topic 3
- The end goal of this is e.g. Identify which jobs meet our expectations of years experience and total salary.

### 📘 Concepts Covered

General concepts we’re going to cover

- Concept 1
- Concept 2
- Concept 3

## LAG

### 📝 Notes

- `LAG()`: Retrieves data from a previous row in the same result set.

### 💻 Final Result

- Describe what the final result should be e.g. return the retention by X cohort.

#### Problem Description

**`FUNCTION` / Concept Covered**

1. Go into specific step / what we’re going to do. E.g. Use the `=` operator to set a new column to be equal to Experience

## LEAD

### 📝 Notes

- `LEAD()`: Retrieves data from the following row in the same result set.

### 💻 Final Result

- Describe what the final result should be e.g. return the retention by X cohort.

#### Problem Description

**`FUNCTION` / Concept Covered**

1. Go into specific step / what we’re going to do. E.g. Use the `=` operator to set a new column to be equal to Experience

Scenario 1: Use LAG() to calculate month-over-month revenue change.

In [ ]:
%%sql

SELECT
    DATE_TRUNC('month', s.OrderDate) AS sales_month,
    SUM(s.SalesAmount) AS total_revenue,
    LAG(SUM(s.SalesAmount)) OVER (ORDER BY DATE_TRUNC('month', s.OrderDate)) AS previous_month_revenue,
    SUM(s.SalesAmount) - LAG(SUM(s.SalesAmount)) OVER (ORDER BY DATE_TRUNC('month', s.OrderDate)) AS revenue_change
FROM
    Sales s
GROUP BY
    DATE_TRUNC('month', s.OrderDate)
ORDER BY
    sales_month;


Scenario 2: Use LEAD() to predict next month’s revenue.

In [ ]:
%%sql

SELECT
    DATE_TRUNC('month', s.OrderDate) AS sales_month,
    SUM(s.SalesAmount) AS total_revenue,
    LEAD(SUM(s.SalesAmount)) OVER (ORDER BY DATE_TRUNC('month', s.OrderDate)) AS next_month_revenue
FROM
    Sales s
GROUP BY
    DATE_TRUNC('month', s.OrderDate)
ORDER BY
    sales_month;


Scenario 3: Use FIRST_VALUE() to find the first recorded sale for each product category.

In [ ]:
%%sql

SELECT
    pc.ProductCategoryName AS category,
    p.ProductName AS product,
    FIRST_VALUE(s.OrderDate) OVER (PARTITION BY pc.ProductCategoryName ORDER BY s.OrderDate) AS first_sale_date
FROM
    Sales s
JOIN
    Products p ON s.ProductKey = p.ProductKey
JOIN
    ProductCategories pc ON p.ProductCategoryKey = pc.ProductCategoryKey
ORDER BY
    category, first_sale_date;
